In [ ]:
from sklearn.datasets import make_classification 
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from skore import CrossValidationReport

pipe = make_pipeline(StandardScaler(), LogisticRegression())
X, y = make_classification(n_samples=16000, n_features=10)
report = CrossValidationReport(pipe, X, y)

In [ ]:
from skore import render_model_card

render_model_card(
    report,
    name="Demo classifier", 
    description="This is really just a demo of a model card. The model is really silly.",
    intended_use="The goal of this pipeline is to show a bunch of properties that can be displayed in a model card.",
    limitations="There is no reason to ever use this system. Ever."
)